In [1]:
from bubblewrap import Bubblewrap
import numpy as np
from ava.models.vae import VAE,X_SHAPE
from ava.models.window_vae_dataset import get_window_partition, get_fixed_window_data_loaders
from ava.preprocessing.utils import get_spec
from ava.preprocessing.preprocess import tune_window_preprocessing_params,get_audio_filenames
import os
import glob
from scipy.io import wavfile

Jax plugin configuration error: Exception when calling jax_plugins.xla_cuda12.initialize()
Traceback (most recent call last):
  File "/home/miles/.local/lib/python3.10/site-packages/jax/_src/xla_bridge.py", line 438, in discover_pjrt_plugins
    plugin_module.initialize()
  File "/home/miles/.local/lib/python3.10/site-packages/jax_plugins/xla_cuda12/__init__.py", line 78, in initialize
    options = xla_client.generate_pjrt_gpu_plugin_options()
AttributeError: module 'jaxlib.xla_client' has no attribute 'generate_pjrt_gpu_plugin_options'


## Load data for VAE

In [2]:
dataPath = '/home/miles/Downloads/goldberg_budgie_data'
bird = 'Bird2CU22001'
conditions = ['*PBS_t1']
fullPath = os.path.join(dataPath,bird)
fullPaths = sum([glob.glob(os.path.join(fullPath,c)) for c in conditions],[])

wavs = [glob.glob(os.path.join(p,'*.wav')) for p in fullPaths]
times = [glob.glob(os.path.join(p,'*.time.txt')) for p in fullPaths]
for w,t in zip(wavs,times):
    w.sort()
    t.sort()

fs,_ = wavfile.read(wavs[0][0])

In [5]:
p = {
	'fs': fs,
	'get_spec': get_spec,
	'num_freq_bins': X_SHAPE[0],
	'num_time_bins': X_SHAPE[1],
	'nperseg': 512, # FFT
	'noverlap': 256, # FFT
	'max_dur': 1e9, # Big number
	'window_length': 0.12,
	'min_freq': 400,
	'max_freq': 10e3,
	'spec_min_val': 2.0,
	'spec_max_val': 6.5,
	'mel': True, # Frequency spacing
	'time_stretch': False,
	'within_syll_normalize': False,
	'real_preprocess_params': ('min_freq', 'max_freq', 'spec_min_val', \
			'spec_max_val'),
	'int_preprocess_params': tuple([]),
	'binary_preprocess_params': ('mel', 'within_syll_normalize'),
}

p = tune_window_preprocessing_params(fullPaths, p)
#afs = get_audio_filenames(fullPaths[0])
#afs = np.array(afs)
#file_index = np.random.randint(len(afs))
#audio_filename = afs[file_index]
#fs, audio = wavfile.read(audio_filename)
#duration = len(audio) / fs
#print(audio.shape)

Tune preprocessing parameters:


Set value for min_freq: [400]  
Set value for max_freq: [10000.0]  
Set value for spec_min_val: [2.0]  
Set value for spec_max_val: [6.5]  
Set value for mel: [T]  
Set value for within_syll_normalize: [F]  
Continue? [y] or [s]top tuning or [r]etune params:  y
Continue? [y] or [s]top tuning or [r]etune params:  y
Continue? [y] or [s]top tuning or [r]etune params:  y
Continue? [y] or [s]top tuning or [r]etune params:  y
Continue? [y] or [s]top tuning or [r]etune params:  y
Continue? [y] or [s]top tuning or [r]etune params:  r
Set value for min_freq: [400]  
Set value for max_freq: [10000.0]  20000
Set value for spec_min_val: [2.0]  1.
Set value for spec_max_val: [6.5]  4.5
Set value for mel: [T]  
Set value for within_syll_normalize: [F]  
Continue? [y] or [s]top tuning or [r]etune params:  y
Continue? [y] or [s]top tuning or [r]etune params:  y
Continue? [y] or [s]top tuning or [r]etune params:  y
Continue? [y] or [s]top tuning or [r]etune params:  y
Continue? [y] or [s]top tuning or 

In [14]:
def convert_txt_labels(roi_files,fs):

    new_files=[]

    for file in roi_files:
        try:
            on,off = np.loadtxt(file,delimiter=',',unpack=True)
            on,off = np.round(on/fs,4),np.round(off/fs,4)
            combined = np.stack([on,off]).T
            header = "Onsets/offsets for " + file
            seg_fn = file.split('.txt')[0] + 'combined.txt'
            np.savetxt(seg_fn, combined, fmt='%.5f', header=header)

            new_files.append(seg_fn)
        except:
            print(file)
            raise NotImplementedError
    return new_files

In [15]:
new_files = convert_txt_labels(times[0],fs)

In [20]:
partition = get_window_partition(fullPaths, fullPaths, 1,roi_suffix='.timecombined.txt')
partition['test'] = partition['train']
num_workers = min(7, os.cpu_count()-1)
loaders = get_fixed_window_data_loaders(partition, p, \
	num_workers=num_workers, batch_size=128,save_memory=False)
loaders['test'] = loaders['train']

In [23]:
model = VAE(save_dir='/home/miles/Downloads/goldberg_budgie_data/single_bird_single_session_VAE')
model.train_loop(loaders, epochs=2000, test_freq=100,save_freq=250,vis_freq=100)

Training: epochs 0 to 1999
Training set: 2048
Test set: 2048
Epoch: 0 Average loss: 65680.0706
Test loss: 940.1427
Epoch: 1 Average loss: 7752.8421
Epoch: 2 Average loss: 1947.4272
Epoch: 3 Average loss: 1068.1038
Epoch: 4 Average loss: 902.0843
Epoch: 5 Average loss: 672.2601
Epoch: 6 Average loss: 550.1462
Epoch: 7 Average loss: 537.3779
Epoch: 8 Average loss: 476.9619
Epoch: 9 Average loss: 512.3916
Epoch: 10 Average loss: 437.4984
Epoch: 11 Average loss: 443.4133
Epoch: 12 Average loss: 437.9591
Epoch: 13 Average loss: 436.8210
Epoch: 14 Average loss: 413.4246
Epoch: 15 Average loss: 390.0253
Epoch: 16 Average loss: 385.2843
Epoch: 17 Average loss: 358.9944
Epoch: 18 Average loss: 365.0661
Epoch: 19 Average loss: 329.1723
Epoch: 20 Average loss: 378.3087
Epoch: 21 Average loss: 355.1899
Epoch: 22 Average loss: 325.3222
Epoch: 23 Average loss: 382.4850
Epoch: 24 Average loss: 320.6254
Epoch: 25 Average loss: 323.5555
Epoch: 26 Average loss: 374.9348
Epoch: 27 Average loss: 309.6915


## To-do: 
- [ ] embed vocalization as trajectories
- [ ] fit bubblewrap
- [ ] estimate fisher
- [ ] estimate fisher on toy data

In [ ]:
fullPaths = sum([],[glob.glob(os.path.join(fullPath,c)) for c in conditions])
print(os.path.join(fullPath,conditions[0]))
print(os.path.isdir(fullPath))
print(glob.glob(os.path.join(fullPath,conditions[0])))